# Adaptive PDE discretizations on Cartesian grids
## Volume : Reproducible research
## Part : Eikonal CPU/GPU solvers comparison
## Chapter : Manifolds defined by local charts

We solve eikonal PDEs on manifolds defined by local charts. In particular the projective plane, and the klein bottle.

This functionality is so far only implemented in the GPU eikonal solver. It is used in the context of the rolling ball models, which involve PDEs on the group $\mathrm{SO}_3$, parametrized by three dimensional projective space $\mathrm{P}_3$.

[**Summary**](Summary.ipynb) of volume Reproducible research, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Projective plane](#1.-Projective-plane)
    * [1.1 Manifold description](#1.1-Manifold-description)
    * [1.2 Running the HFM library](#1.2-Running-the-HFM-library)
    * [1.3 Comparison with exact solutions.](#1.3-Comparison-with-exact-solutions.)
    * [1.4 Variants in the computation](#1.4-Variants-in-the-computation)
    * [1.5 Glue along a narrow band](#1.5-Glue-along-a-narrow-band)
    * [1.6 Built-in construction of a projective space](#1.6-Built-in-construction-of-a-projective-space)
  * [2. The sphere](#2.-The-sphere)
    * [2.1 Manifold description](#2.1-Manifold-description)
    * [2.2 Solving the eikonal equation](#2.2-Solving-the-eikonal-equation)
    * [2.3 Comparison with the exact solution](#2.3-Comparison-with-the-exact-solution)
    * [2.4 Built-in construction](#2.4-Built-in-construction)
  * [3. Circular band](#3.-Circular-band)
  * [4. The Moëbius strip](#4.-The-Moëbius-strip)



**Acknowledgement.** Some of the experiments presented in these notebooks are part of 
ongoing research with Ludovic Métivier and Da Chen.

Copyright Jean-Marie Mirebeau, Centre Borelli, ENS Paris-Saclay, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Chart_GPU','Repro'))

In [2]:
from agd import Eikonal
from agd import AutomaticDifferentiation as ad
from agd import FiniteDifferences as fd
from agd import Metrics
from agd import LinearParallel as lp
from agd import Sphere as sp
norm = ad.Optimization.norm

In [3]:
import numpy as np; xp=np; allclose = np.allclose
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global Eikonal
    Eikonal, = rreload([Eikonal],"../..")
    Eikonal.dictIn.default_mode = 'gpu'

### 0.1 Additional configuration

The GPU accelerated eikonal solver is mandatory for this notebook.

In [5]:
if ad.cupy_generic.cp is None: raise ad.DeliberateNotebookError("cupy module not found")

DeliberateNotebookError: cupy module not found

In [6]:
xp,Eikonal,plt,allclose = [ad.cupy_friendly(e) for e in (xp,Eikonal,plt,allclose)]

Replacing numpy with cupy, set to output 32bit ints and floats by default.
Using cp.asarray(*,dtype=np.float32) as the default caster in ad.array.
Setting dictIn.default_mode = 'gpu' in module agd.Eikonal .
Returning a copy of module matplotlib.pyplot whose functions accept cupy arrays as input.
Setting float32 compatible default values atol=rtol=1e-5 in np.allclose


## 1. Projective plane



### 1.1 Manifold description

The projective plane is defined as the unit sphere, in which pairs of opposite points are identified.
The `sphere_from_plane` map parametrizes the northern hemisphere from the equatorial disk, by projection from the south pole.

In [7]:
def chart_mapping(x): 
    """
    Glue map for the projective plane, as parameterized from the equator.
    (Maps a point to the other point with which it identifies.)
    """    
    return sp.plane_from_sphere( - sp.sphere_from_plane(x))

The equatorial projection is conformal to the isotropic metric on the plane, with an explicit cost function.

In [8]:
def metric(x):
    """
    Pulls back the Euclidean metric on the sphere onto the plane.
    """
    x_ad = ad.Dense.identity(constant=x,shape_free=(len(x),))
    Jac = np.moveaxis(sphere_from_plane(x_ad).gradient(),0,1)
    return lp.dot_AA(lp.transpose(Jac),Jac)

def cost(x):
    """
    Cost function associated with the conformal metric of the stereographic projection.
    """
    return np.sqrt(metric(x)[0,0]) # Alternatively, use the explicit expression 2/(1+ |x|^2)

In [9]:
def cost(x):
    """
    Isotropic cost function for the projective plane, as parametrized from the equator.
    """
    return 2/(1.+lp.dot_VV(x,x))

### 1.2 Running the HFM library

We use a square, strictly containing the equatorial unit ball, as parametrization domain.

In [10]:
n = 201
hfmIn = Eikonal.dictIn({
    'model':'Isotropic2',
    'seed':[0.5,0.7],
    'exportValues':True,
    'factoringRadius':10,
})

r=1.1
hfmIn.SetRect([[-r,r],[-r,r]],dimx=n)
X = hfmIn.Grid()

hfmIn['cost']=cost(X)
hfmIn.SetUniformTips((4,4))

Next we need to describe the local charts.

In [11]:
hfmIn['chart_mapping'] = chart_mapping(X) # mapping to opposite point
hfmIn['chart_jump'] = norm(X,axis=0) > (1+r)/2 # When geodesics should jump
#hfmIn['chart_nitermax'] = 1

In [12]:
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.03091716766357422 seconds, and 74 iterations.
Post-Processing


The computed distance values are minimal at the seed, and at the opposite point. 

In [13]:
plt.title("Distance values in the projective plane")
plt.contourf(*X,hfmOut['values'])
plt.axis('equal'); plt.colorbar();

The geodesics jump to according to the local chart as prescribed.

In [14]:
plt.title("Geodesics in the projective plane")
plt.axis('equal')
plt.contourf(*X,hfmIn['chart_jump'])
for geo in hfmOut['geodesics']: plt.plot(*geo)

### 1.3 Comparison with exact solutions.

The geodesics and geodesic distances on the sphere are known, and correspond to paths along the great circles.
The situation is similar on the projective plane, except closed geodesics are only half circles.

In [15]:
def spherical_distance(p,q):
    """Distance on the sphere, betweeen two points of the sphere"""
    p,q = fd.common_field((p,q),depths=(1,1))
    return 2*np.arcsin(norm(p-q,axis=0)/2)

In [16]:
seed_S = sp.sphere_from_plane(hfmIn['seed'])
X_S = sp.sphere_from_plane(X)
exact = np.minimum(spherical_distance(seed_S,X_S),spherical_distance(-seed_S,X_S))

In [17]:
plt.title('Exact distance values')
plt.contourf(*X,exact)
plt.colorbar(); plt.axis('equal');

The values are quite close. The biggest discrepancy is in the center of the domain, where the metric is stronger. In particular, it is unrelated with the chart mapping.

In [18]:
assert norm(exact-hfmOut['values'],ord=np.inf) < 0.02

In [19]:
plt.contourf(*X,exact-hfmOut['values'])
plt.colorbar(); plt.axis('equal');

### 1.4 Variants in the computation

Multi-precision is supported. This option improves accuracy in some very large test cases.

In [20]:
hfmIn['multiprecision']=True
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.024010419845581055 seconds, and 68 iterations.
Post-Processing


In [21]:
assert norm(exact-hfmOut['values'],ord=np.inf) < 0.02
hfmIn.pop('multiprecision',None);

### 1.5 Glue along a narrow band

One may want to apply the boundary conditions only in a narrow band.

In [22]:
radius = norm(X,axis=0)
narrow_band = np.logical_and(0.9<=radius,radius<=1.1)
hfmIn['chart_mapping'] = np.where(narrow_band,chart_mapping(X),np.nan)
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.023935317993164062 seconds, and 78 iterations.
Post-Processing


Numerical error remains untouched within the fundamental domain, although larger errors are produced elsewhere.

In [23]:
assert norm(np.where(norm(X,axis=0)<1,hfmOut['values']-exact,0),ord=np.inf) < 0.02 # Same error in fundamental domain
norm(hfmOut['values']-exact,ord=np.inf) # Larger error outside

array(0.119, dtype=float32)

### 1.6 Built-in construction of a projective space

Projective spaces are fairly common manifolds, hence the construction presented above is built-in the agd library for convenience.

In [24]:
hfmIn = Eikonal.dictIn({
    'model':'Isotropic2',
    'seed':[0.5,0.7],
    'exportValues':True,
    'factoringRadius':10,
})

In [25]:
transformations = hfmIn.SetSphere(n,separation=False)

In [26]:
X = hfmIn.Grid()
hfmIn['cost'] = 2./(1+lp.dot_VV(X,X)) # Conformal isotropic cost
hfmIn.SetUniformTips((4,4))

In [27]:
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.02396702766418457 seconds, and 74 iterations.
Post-Processing


In [28]:
assert norm(exact-hfmOut['values'],ord=np.inf) < 0.02 # Same solution is computed

The `SetSphere` method returns the projection functions from the equatorial plane to the sphere and conversely.

In [29]:
transformations

{'from_sphere': <function agd.Eikonal.DictIn_detail.proj_fromsphere(q)>,
 'to_sphere': <function agd.Eikonal.DictIn_detail.proj_tosphere(x)>}

In these mapped coordinates, geodesics are arcs of circles on the sphere, with occasional jumps from a point to the opposite one.

In [30]:
plt.axes(projection='3d')
for geo in hfmOut['geodesics']: 
    geo = transformations['to_sphere'](geo)
    plt.plot(*geo)

## 2. The sphere



### 2.1 Manifold description

The sphere can be described using two local charts

In [36]:
def chart_mapping(x):
    x=x.copy()
    s = np.sign(x[0])
    x[0] -= s*r
    y = sp.plane_from_sphere(-sp.sphere_from_plane(x))
    y[:,np.abs(y[0])>r]=np.nan
    y[0] -= s*r
    return y

def cost(x):
    x=x.copy()
    s = np.sign(x[0])
    x[0] -= s*r
    return 2/(1+lp.dot_VV(x,x))

### 2.2 Solving the eikonal equation

In [37]:
r=1.1

hfmIn = Eikonal.dictIn({
    'model':'Isotropic2',
    'seed':[0.5-r,0.7],
    'exportValues':True,
    'factoringRadius':10,
})

hfmIn.SetRect([[-2*r,2*r],[-r,r]],dimx=401) # Twice larger domain along x axis

X = hfmIn.Grid()
hfmIn.SetUniformTips((8,4))

In [38]:
hfmIn['cost']=cost(X)
hfmIn['chart_jump'] = cost(X)<0.95
hfmIn['chart_nitermax']=10

**Important : set `chart_mapping` to `NaN` in the `walls` location.** Otherwise, some solution values will be pasted in the walls location, according to the `chart_mapping`. Then, they may be pasted elsewhere in a subsequent call, corrupting the solution values.

In [39]:
hfmIn['walls']=np.abs(X[0])<0.02 # Separation between the two charts
hfmIn['chart_mapping'] = np.where(hfmIn['walls'],np.nan,chart_mapping(X)) # mapping to opposite point

In [40]:
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.043752193450927734 seconds, and 132 iterations.
Post-Processing


In [41]:
plt.figure(figsize=(16,7))
plt.title('Solution values, with the two seed points')
plt.contourf(*X,hfmOut['values'])
plt.axis('equal'); 
plt.colorbar();
plt.scatter(*hfmIn['seed'])
plt.scatter(*chart_mapping(hfmIn['seed']));

In [42]:
plt.axis('equal')
for geo in hfmOut['geodesics']: plt.plot(*geo)

### 2.3 Comparison with the exact solution

In [44]:
def to_sphere(x):
    x=x.copy()
    s=np.sign(x[0])
    x[0]-=r*s
    return sp.sphere_from_plane(x)*s
    
exact = spherical_distance(to_sphere(X),to_sphere(hfmIn['seed']))

In [45]:
error = np.where(hfmIn['walls'],0.,hfmOut['values']-exact)
assert norm(error,ord=np.inf) < 0.03 # Errors are quite small

### 2.4 Built-in construction

Spheres are common manifolds in applications, hence the agd library provides a built-in construction.

In [67]:
hfmIn0 = hfmIn; XGrid0 = X

In [68]:
hfmIn = Eikonal.dictIn({
    'model':'Isotropic2',
    'seed':[0.5-r,0.7],
    'exportValues':True,
    'factoringRadius':10,
})

transformations = hfmIn.SetSphere(n) #,separation=1)
hfmIn.SetUniformTips((8,4))

In [69]:
XGrid = hfmIn.Grid()
X,chart = transformations['from_grid'](XGrid) 
hfmIn['cost'] = 2./(1.+lp.dot_VV(X,X))
hfmIn['walls'] = np.isnan(hfmIn['cost'])

In [70]:
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.03792905807495117 seconds, and 105 iterations.
Post-Processing


In [71]:
plt.axis('equal')
for geo in hfmOut['geodesics']: plt.plot(*geo)

In [72]:
plt.contourf(*XGrid,hfmOut['values'])
plt.axis('equal');

In [73]:
plt.figure(figsize=(9,8))
plt.axes(projection='3d')
for geo in hfmOut['geodesics']: 
    geo = transformations['to_sphere'](*transformations['from_grid'](geo))
    plt.plot(*geo)

## 3. Circular band

We implement periodicity along one axis. The point is to illustrate that `chart_mapping` can be broadcasted over the first coordinates, which is helpful in the case of spherical or projective bundles over a domain, such as $R^3 \times S^2$ or $R^2 \times P^3$ which are often considered in other notebooks. Broadcasting avoids storing an unnecessary large ` chart_mapping`.

**The `periodic` field**. Periodicity along the second axis may also be achieved in a simpler and more computationally efficient fashion, by introducing `'periodic':[False,True]` in `hfmIn`.

In [53]:
r=1.1
hfmIn = Eikonal.dictIn({
    'model':'Isotropic2',
    'seed':[0.5,0.3],
    'exportValues':True,
    'cost':1,
    'factoringRadius':10,
})
hfmIn.SetRect([[-1,1],[-r,r]],dimx=100)
hfmIn.SetUniformTips((4,4))
_,aY = hfmIn.Axes()
hfmIn['chart_mapping'] = np.expand_dims(np.where(aY>0,aY-2,aY+2),axis=0) # periodicity along y axis
hfmIn['chart_jump'] = np.abs(aY)> (1+r)/2

X = hfmIn.Grid()

In [54]:
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.0069811344146728516 seconds, and 19 iterations.
Post-Processing


In [55]:
plt.contourf(*X,hfmOut['values'])
plt.axis('equal'); plt.colorbar();

In [56]:
def euclidean_distance(p,q): 
    p,q = fd.common_field((p,q),depths=(1,1))
    return norm(p-q,axis=0)

exact = np.minimum( euclidean_distance(X,hfmIn['seed']), euclidean_distance(X,hfmIn['seed'] - xp.array([0,2])) )

In [57]:
assert norm(exact-hfmOut['values'],ord=np.inf) < 0.02

## 4. The Moëbius strip

In [58]:
r=1.1
hfmIn = Eikonal.dictIn({
    'model':'Isotropic2',
    'seed':[0.5,0.3],
    'exportValues':True,
    'cost':1,
    'factoringRadius':10,
})
hfmIn.SetRect([[-1,1],[-r,r]],dimx=100)
hfmIn.SetUniformTips((4,4))
_,aY = hfmIn.Axes()
hfmIn['chart_jump'] = np.abs(aY)> (1+r)/2

X = hfmIn.Grid()

In [59]:
def chart_mapping(x):
    s = np.sign(x[1])
    return [-x[0],x[1]-2*s]

hfmIn['chart_mapping'] = chart_mapping(X)

In [60]:
hfmOut = hfmIn.Run()

Setting the kernel traits.
Preparing the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.0059854984283447266 seconds, and 19 iterations.
Post-Processing


In [61]:
plt.contourf(*X,hfmOut['values'])
plt.axis('equal');